In [45]:
from langchain import PromptTemplate
import pinecone
import os
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader



In [ ]:
#make .env file in base directory and Set the PINECONE API KEY and PINECONE API ENV as PINECONE_API_KEY and PINECONE_API_ENV as environment variable to use here
PINECONE_API_KEY = "PINECONE_API_KEY"
PINECONE_API_ENV = "PINECONE_API_ENV"

In [21]:
#Extract data from the PDF files
def load_pdffiles(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [22]:
pdfdata = load_pdffiles("E:\workspace\data")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Rajesh Kumar\AppData\Local\Temp\ipykernel_22972\2664242978.py:1: SyntaxWarning: invalid escape sequence '\w'
  pdfdata = load_pdffiles("E:\workspace\data")


In [23]:
#pdfdata

In [ ]:
#Create text chunks
def text_split(pdfdata):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(pdfdata)

    return text_chunks

In [25]:
text_chunks = text_split(pdfdata)
print("length of my chunk:", len(text_chunks))

length of my chunk: 498


In [ ]:
#text_chunks

[Document(metadata={'source': 'E:\\workspace\\data\\gstr3Bofflineutility.pdf', 'page': 0}, page_content='GSTR3B Offline Utility (Last Updated on: 16/10/2017) \nDesigned and Developed by GSTN  Page 1 of 24 \n \n \n \n  \nFAQs and User Manual – GSTR3B Offline \nUtility \n \n \nGoods and Services Tax Network'),
 Document(metadata={'source': 'E:\\workspace\\data\\gstr3Bofflineutility.pdf', 'page': 1}, page_content='GSTR3B Offline Utility (Last Updated on: 16/10/2017) \nDesigned and Developed by GSTN  Page 2 of 24 \n \nTable of Contents \n1. FAQs on GSTR3B Offline Utility .......................................................................................... 3 \n2. GST3B Offline Utility: An Overview ..................................................................................... 4 \n3. Downloading the GSTR3B Offline Utility ............................................................................ 5'),
 Document(metadata={'source': 'E:\\workspace\\data\\gstr3Bofflineutility.pdf', 

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [29]:
embeddings = download_hugging_face_embeddings()

C:\Users\Rajesh Kumar\AppData\Local\Temp\ipykernel_22972\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
e:\workspace\env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rajesh Kumar\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setti

In [30]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [34]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [36]:
 #query_result

In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
                      environment=PINECONE_API_ENV)

#create index on pinecone and pass the same name here
index_name="gst-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing in pinecore vector db
docsearch=pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

